In [1]:
# Helper functions
from pathlib import Path
from copy import deepcopy
from typing import Tuple, List, Dict

import numpy as np
import open3d as o3d
from transforms3d.quaternions import quat2mat, mat2quat

import rosbag
from cv_bridge import CvBridge
from geometry_msgs.msg import Transform

def transform_msg2mat(transform: Transform) -> np.ndarray:
    """Convert a geometry_msgs/Transform to a 4x4 transformation matrix
    :param transform: a geometry_msgs.msg.Transform.
    """
    p = transform.translation
    q = transform.rotation
    
    T = np.eye(4)
    T[:3, :3] = quat2mat([q.w, q.x, q.y, q.z])  # [w, x, y, z]
    T[:3, 3] = [p.x, p.y, p.z]
    return T

def read_frame_data_from_bag(bag: rosbag.Bag) -> Dict[str, list]:
    """Read frame data from a rosbag"""
    frame_data = {
        'rgb_imgs': [],  # [H, W, 3] uint8 np.ndarray
        'rgb_img_timestamps': [],
        'rgb_intrinsics': [],  # [3, 3] float64 np.ndarray
        'rgb_intr_timestamps': [],
        'depth_imgs': [],  # [H, W] uint16 np.ndarray
        'depth_img_timestamps': [],
        'depth_intrinsics': [],  # [3, 3] float64 np.ndarray
        'depth_intr_timestamps': [],
        'poses': [],  # T_world_cam, [4, 4] float64 np.ndarray
        'pose_timestamps': [],
        'timestamps': [],  # unique timestamps with imgs and poses
    }
    
    cvbridge = CvBridge()
    topic_msgs = bag.read_messages(topics=['/tf',
                                           '/camera/color/image_raw',
                                           '/camera/color/camera_info',
                                           '/camera/aligned_depth_to_color/image_raw',
                                           '/camera/aligned_depth_to_color/camera_info'])
    for i, (topic, msg, t) in enumerate(topic_msgs):
        # Extract msg timestamp
        if hasattr(msg, 'header'):
            msg_timestamp = round(msg.header.stamp.to_sec(), 6)
        else:  # topic == '/tf'
            msg_timestamp = round(msg.transforms[0].header.stamp.to_sec(), 6)

        if topic == '/tf' and msg.transforms[0].header.frame_id == "map" \
                and msg.transforms[0].child_frame_id == 'imu':
            frame_data['poses'].append(transform_msg2mat(msg.transforms[0].transform))
            frame_data['pose_timestamps'].append(msg_timestamp)
        elif topic == '/camera/color/image_raw':
            frame_data['rgb_imgs'].append(cvbridge.imgmsg_to_cv2(msg, desired_encoding='rgb8'))
            frame_data['rgb_img_timestamps'].append(msg_timestamp)
        elif topic == '/camera/color/camera_info':
            frame_data['rgb_intrinsics'].append(np.asanyarray(msg.P).reshape(3, 4)[:3, :3])
            frame_data['rgb_intr_timestamps'].append(msg_timestamp)
        elif topic == '/camera/aligned_depth_to_color/image_raw':
            frame_data['depth_imgs'].append(cvbridge.imgmsg_to_cv2(msg, desired_encoding='16UC1'))
            frame_data['depth_img_timestamps'].append(msg_timestamp)
        elif topic == '/camera/aligned_depth_to_color/camera_info':
            frame_data['depth_intrinsics'].append(np.asanyarray(msg.P).reshape(3, 4)[:3, :3])
            frame_data['depth_intr_timestamps'].append(msg_timestamp)

    # Find timestamps associations
    from functools import reduce
    timestamps = reduce(np.intersect1d, [v for k, v in frame_data.items() if '_timestamps' in k]).tolist()
    frame_data['timestamps'] = timestamps
    print(f"Found {len(timestamps)} timestamps with images and poses")
    
    return frame_data

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2022-05-30 17:41:05,469 - _logger - Log opened: Tue May 31 00:41:05 2022 UTC
INFO - 2022-05-30 17:41:05,502 - topics - topicmanager initialized


In [2]:
# dense reconstruction
#bagpath = Path('/maya-slam/slam_algorithms/rosbags/d435i_bag_2022-05-20-22-23-07_posed.bag').resolve()
bagpath = Path('/maya-slam/slam_algorithms/rosbags/d435i_bag_2022-05-20-22-25-23_posed.bag').resolve()
bag = rosbag.Bag(bagpath)

# Read frame data from rosbag
frame_data = read_frame_data_from_bag(bag)

Found 1471 timestamps with images and poses


In [3]:
frame_data.keys()

dict_keys(['rgb_imgs', 'rgb_img_timestamps', 'rgb_intrinsics', 'rgb_intr_timestamps', 'depth_imgs', 'depth_img_timestamps', 'depth_intrinsics', 'depth_intr_timestamps', 'poses', 'pose_timestamps', 'timestamps'])

In [4]:
import open3d.core as o3c

# Reference:
# http://www.open3d.org/docs/0.15.1/tutorial/t_reconstruction_system/dense_slam.html

#device = o3c.Device('CUDA:0')
device = o3c.Device('CPU:0')

T_world_cam = o3c.Tensor(np.identity(4))
model = o3d.t.pipelines.slam.Model(
    0.01,   # voxel_size
    16,     # block_resolution
    50000,  # block_count
    transformation=T_world_cam,
    device=device
)
model

In [5]:
def get_frame_data_at_timestamp(frame_data: Dict[str, list], timestamp: float):
    """Get frame data at timestamp"""
    rgb_im = frame_data['rgb_imgs'][frame_data['rgb_img_timestamps'].index(timestamp)]
    rgb_K = frame_data['rgb_intrinsics'][frame_data['rgb_intr_timestamps'].index(timestamp)]
    
    depth_im = frame_data['depth_imgs'][frame_data['depth_img_timestamps'].index(timestamp)]
    depth_K = frame_data['depth_intrinsics'][frame_data['depth_intr_timestamps'].index(timestamp)]
    
    np.testing.assert_allclose(rgb_K, depth_K,
                               err_msg='Different intrinsic for rgb and depth, possibly misaligned')
    K = rgb_K
    
    cam_pose = frame_data['poses'][frame_data['pose_timestamps'].index(timestamp)]\
    
    return rgb_im, depth_im, K, cam_pose

In [6]:
from tqdm import tqdm

start_idx = 100

cam_poses = []
input_frame, raycast_frame = None, None

for i, timestamp in enumerate(tqdm(frame_data['timestamps'][start_idx:])):
    rgb_im, depth_im, K, cam_pose = get_frame_data_at_timestamp(frame_data, timestamp)
    height, width, channels = rgb_im.shape
    
    rgb_im = o3d.t.geometry.Image(rgb_im).to(device)
    depth_im = o3d.t.geometry.Image(depth_im).to(device)
    K = o3c.Tensor(K)
    extrinsic = o3c.Tensor(np.linalg.inv(cam_pose))
    
    # Initialization
    if input_frame is None and raycast_frame is None:
        input_frame = o3d.t.pipelines.slam.Frame(depth_im.rows, depth_im.columns,
                                                 K, device)
        raycast_frame = o3d.t.pipelines.slam.Frame(depth_im.rows, depth_im.columns,
                                                   K, device)
    
    input_frame.set_data_from_image('depth', depth_im)
    input_frame.set_data_from_image('color', rgb_im)
    
    if i > 0:
        result = model.track_frame_to_model(input_frame, raycast_frame,
                                            depth_scale=1e3,
                                            depth_max=10.0,
                                            depth_diff=0.07)
        T_world_cam = T_world_cam @ result.transformation
    
    cam_poses.append(T_world_cam.cpu().numpy())
    model.update_frame_pose(i, T_world_cam)
    model.integrate(input_frame, depth_scale=1e3, depth_max=10.0,
                    trunc_voxel_multiplier=8.0)
    model.synthesize_model_frame(raycast_frame, depth_scale=1e3, depth_min=0.1, depth_max=10.0,
                                 trunc_voxel_multiplier=8.0, enable_color=True)


100%|██████████████████████████████████████████████████████████████████████████████████████████| 1371/1371 [03:43<00:00,  6.13it/s]


In [11]:
# Visualize pose traj
import plotly.express as px

df = {'method': [], 'pos_x': [], 'pos_y': [], 'pos_z': [], 'timestamp': []} # , 'delta_t': []

### o3d dense RGBD slam ###
for i, pose in enumerate(cam_poses):
    df['method'].append('Open3D Dense RGB-D SLAM')
    df['timestamp'].append(i)
    df['pos_x'].append(pose[0, 3])
    df['pos_y'].append(pose[1, 3])
    df['pos_z'].append(pose[2, 3])
    
### MapLab ###
# posed images from MapLab
import rosbag
import numpy as np
from pathlib import Path

#bagpath = Path('../poseimage.bag').resolve()
bag = rosbag.Bag(bagpath)
topic_msgs = bag.read_messages(topics=['/tf'])

for i, (topic, msg, t) in enumerate(topic_msgs):
    msg_timestamp = round(msg.transforms[0].header.stamp.to_sec(), 6)
    
    if msg.transforms[0].header.frame_id == "map" and msg.transforms[0].child_frame_id == 'imu':
        #print(topic, msg, t)
        df['method'].append('MapLab')
        df['timestamp'].append(msg_timestamp)
        df['pos_x'].append(msg.transforms[0].transform.translation.x)
        df['pos_y'].append(msg.transforms[0].transform.translation.y)
        df['pos_z'].append(msg.transforms[0].transform.translation.z)

df = {k: np.array(v) for k, v in df.items()}       
fig = px.line_3d(df, x='pos_x', y='pos_y', z='pos_z', color='method',
                 hover_data=['timestamp'], markers=True,
                 height=720, width=950, title=f'Kitchen {bagpath.name} w/ color')
fig.show()

In [9]:
# Visualize pose traj (w/o color)
import plotly.express as px

df = {'method': [], 'pos_x': [], 'pos_y': [], 'pos_z': [], 'timestamp': []} # , 'delta_t': []

### ORB-SLAM3 ###
for i, pose in enumerate(cam_poses):
    df['method'].append('Open3D Dense RGB-D SLAM')
    df['timestamp'].append(i)
    df['pos_x'].append(pose[0, 3])
    df['pos_y'].append(pose[1, 3])
    df['pos_z'].append(pose[2, 3])
    
### MapLab ###
# posed images from MapLab
import rosbag
import numpy as np
from pathlib import Path

#bagpath = Path('../poseimage.bag').resolve()
bag = rosbag.Bag(bagpath)
topic_msgs = bag.read_messages(topics=['/tf'])

for i, (topic, msg, t) in enumerate(topic_msgs):
    msg_timestamp = round(msg.transforms[0].header.stamp.to_sec(), 6)
    
    if msg.transforms[0].header.frame_id == "map" and msg.transforms[0].child_frame_id == 'imu':
        #print(topic, msg, t)
        df['method'].append('MapLab')
        df['timestamp'].append(msg_timestamp)
        df['pos_x'].append(msg.transforms[0].transform.translation.x)
        df['pos_y'].append(msg.transforms[0].transform.translation.y)
        df['pos_z'].append(msg.transforms[0].transform.translation.z)

df = {k: np.array(v) for k, v in df.items()}       
fig = px.line_3d(df, x='pos_x', y='pos_y', z='pos_z', color='method',
                 hover_data=['timestamp'], markers=True,
                 height=720, width=950)
fig.show()

In [3]:
# helper functions from common.py
def extract_trianglemesh(volume, engine='tensor', file_name=None):
    if engine == 'legacy':
        mesh = volume.extract_triangle_mesh()
        mesh.compute_vertex_normals()
        mesh.compute_triangle_normals()
        if file_name is not None:
            o3d.io.write_triangle_mesh(file_name, mesh)

    elif engine == 'tensor':
        mesh = volume.extract_triangle_mesh(
            weight_threshold=3.0)
        mesh = mesh.to_legacy()

        if file_name is not None:
            o3d.io.write_triangle_mesh(file_name, mesh)

    return mesh

def save_poses(
    path_trajectory,
    poses,
    intrinsic=o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)):
    if path_trajectory.endswith('log'):
        traj = o3d.camera.PinholeCameraTrajectory()
        params = []
        for pose in poses:
            param = o3d.camera.PinholeCameraParameters()
            param.intrinsic = intrinsic
            param.extrinsic = np.linalg.inv(pose)
            params.append(param)
        traj.parameters = params
        o3d.io.write_pinhole_camera_trajectory(path_trajectory, traj)

    elif path_trajectory.endswith('json'):
        pose_graph = o3d.pipelines.registration.PoseGraph()
        for pose in poses:
            node = o3d.pipelines.registration.PoseGraphNode()
            node.pose = pose
            pose_graph.nodes.append(node)
        o3d.io.write_pose_graph(path_trajectory, pose_graph)

In [10]:
volume, cam_poses = model.voxel_grid, cam_poses

volume.save('output.npz')
save_poses('output.log', cam_poses)

NameError: name 'poses' is not defined

In [11]:
save_poses('output.log', cam_poses)

In [ ]:
volume = model.voxel_grid
mesh = extract_trianglemesh(volume)
mesh

In [1]:
import open3d as o3d

volume = o3d.t.geometry.VoxelBlockGrid.load('output.npz')
volume

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
pcd = volume.extract_point_cloud()
pcd

PointCloud on CPU:0 [5395461 points (Float32)] Attributes: colors (dtype = Float32, shape = {5395461, 3}), normals (dtype = Float32, shape = {5395461, 3}).

In [1]:
mesh = extract_trianglemesh(volume)
mesh

NameError: name 'extract_trianglemesh' is not defined

In [8]:
pcd = volume.extract_point_cloud()
pcd

PointCloud on CPU:0 [82819 points (Float32)] Attributes: colors (dtype = Float32, shape = {82819, 3}), normals (dtype = Float32, shape = {82819, 3}).

In [4]:
o3d.visualization.draw_geometries([pcd.to_legacy()])

In [8]:
mesh = model.extract_trianglemesh()
mesh

RuntimeError: [1;31m[Open3D Error] (void open3d::t::geometry::kernel::voxel_grid::ExtractTriangleMeshCUDA(const open3d::core::Tensor&, const open3d::core::Tensor&, const open3d::core::Tensor&, const open3d::core::Tensor&, const open3d::core::Tensor&, const open3d::t::geometry::TensorMap&, open3d::core::Tensor&, open3d::core::Tensor&, open3d::core::Tensor&, open3d::core::Tensor&, open3d::t::geometry::kernel::voxel_grid::index_t, float, float, open3d::t::geometry::kernel::voxel_grid::index_t&) [with tsdf_t = float; weight_t = short unsigned int; color_t = short unsigned int; open3d::t::geometry::kernel::voxel_grid::index_t = int]) /root/Open3D/cpp/open3d/t/geometry/kernel/VoxelBlockGridImpl.h:1327: [MeshExtractionKernel] Unable to allocate assistance mesh structure for Marching Cubes with 93599 active voxel blocks. Please consider using a larger voxel size (currently 0.01) for TSDF integration, or using tsdf_volume.cpu() to perform mesh extraction on CPU.
[0;m

In [8]:
o3d.visualization.draw_geometries([mesh])

In [9]:
o3d.io.write_triangle_mesh('test.ply', mesh)

True

In [14]:
o3d.io

<module 'open3d.cuda.pybind.io'>

In [13]:
o3d.cpu.pybind.io.write_triangle_mesh('test.ply', mesh)

AttributeError: module 'open3d' has no attribute 'cpu'

In [10]:
o3d.t.geometry.Image(depth_im).to(device)

Image[size={480,640}, channels=1, UInt16, CUDA:0]

In [18]:
o3c.Tensor(np.eye(4)).to(o3c.Device('CUDA:0'))

[[1.0 0.0 0.0 0.0],
 [0.0 1.0 0.0 0.0],
 [0.0 0.0 1.0 0.0],
 [0.0 0.0 0.0 1.0]]
Tensor[shape={4, 4}, stride={4, 1}, Float64, CUDA:0, 0x7f65b69e8400]

In [19]:
o3c.det

<function open3d.cuda.pybind.core.PyCapsule.det>